In [5]:
import pandas as pd

In [6]:
filePath_template = "nlp-fairness-for-india-main/templates.tsv"
filePath_religion_idterms = "nlp-fairness-for-india-main/religion_idterms.tsv"
filePath_region_idterms = "nlp-fairness-for-india-main/region_idterms.tsv"
filePath_Caste_idterms = "nlp-fairness-for-india-main/caste_idterms.tsv"
humanDataset_region = "region_stereotypes.tsv"
humanDataset_religion = "religion_stereotypes.tsv"

def getTemplateArray(filePath):
    df_placeolders = pd.read_csv(filePath,sep='\t')
    df_placeolders = df_placeolders.iloc[:,1:]
    df_placeolders = df_placeolders.replace('@','[MASK]',regex=True)
    semiTemplates = df_placeolders.values.tolist()
    templates = [item for sublist in semiTemplates for item in sublist]
    return templates

def getIdentityArray(filePath):
    df_religion = pd.read_csv(filePath,sep='\t')
    religion_list = df_religion.values.tolist()
    religion_idTerms = [item for sublist in religion_list for item in sublist]
    return religion_idTerms

def replaceSlotWithWord(sentences, word):
    return word, [sentence.replace('[SLOT]', word) for sentence in sentences]

def extract_common_tuples(file1_path,file2_path,social_axis):
    file1_df = pd.read_csv(file1_path,sep='\t')
    file2_df = pd.read_csv(file2_path,sep='\t')

    merged_df = pd.merge(file1_df, file2_df, on=[social_axis, 'StereoType'], how='inner')
    merged_df.drop_duplicates(inplace=True)

    return merged_df


In [7]:

# Region
df_regionTuples = pd.read_csv("BERT/regionTuples_MLM.tsv", sep='\t')
regions = df_regionTuples['Region'].unique()
region_dfs = {}
for region in regions:
    region_dfs[region] = df_regionTuples[df_regionTuples['Region'] == region]
for region, region_df in region_dfs.items():
    filePath = "Region_List/" + region + ".tsv"
    region_df.to_csv(filePath,sep='\t',index=False)

# Religion
df_religionTuples = pd.read_csv("BERT/religionTuples_MLM.tsv", sep='\t')
religions = df_religionTuples['Religion'].unique()
religion_dfs = {}
for religion in religions:
    religion_dfs[religion] = df_religionTuples[df_religionTuples['Religion'] == religion]
for religion, religion_df in religion_dfs.items():
    filePath = "Religion_List/" + religion + ".tsv"
    religion_df.to_csv(filePath,sep='\t',index=False)

# Caste
df_casteTuples = pd.read_csv("BERT/casteTuples_MLM.tsv", sep='\t')

sc_st_tuples = df_casteTuples[df_casteTuples['Caste'].str.contains("sc/st", case=False)]
sc_st_file_path = "Caste_List/sc_st_tuples.tsv"
sc_st_tuples.to_csv(sc_st_file_path, sep='\t', index=False)

castes = df_casteTuples['Caste'].unique()
for caste in castes:
    if "sc/st" in caste.lower():
        continue
    caste_df = df_casteTuples[df_casteTuples['Caste'] == caste]
    file_path = f"Caste_List/{caste}.tsv"
    caste_df.to_csv(file_path, sep='\t', index=False)


# Diff Analysis
Analyzing the differences between the tuples generated by the lm and the annotated datasets

In [8]:
# Region
regionIDTerms = getIdentityArray(filePath_region_idterms)
for region in regionIDTerms:
    regionFilePath = "BERT/Region_List/" + region + ".tsv"
    commonRegionTuples_df = extract_common_tuples(regionFilePath,humanDataset_region,"Region")
    saveFilePath = "commonTuples/Region/" + region + ".tsv"
    commonRegionTuples_df.to_csv(saveFilePath,sep='\t',index=False)


# Religion
religionIDTerms = getIdentityArray(filePath_religion_idterms)
for religion in religionIDTerms:
    religionFilePath = "BERT/Religion_List/" + religion + ".tsv"
    commonReligionTuples_df = extract_common_tuples(religionFilePath,humanDataset_religion,"Religion")
    saveFilePath = "commonTuples/Religion/" + religion + ".tsv"
    commonReligionTuples_df.to_csv(saveFilePath,sep='\t',index=False)
